In [ ]:
!pip install google.colab

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Running setup.py clean for pandas
Failed to build pandas
    Running setup.py install for pandas: started
    Running setup.py install for pandas: still running...
    Running setup.py install for pandas: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\conda\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\RVHS-D&T\\AppData\\Local\\Temp\\pip-install-rpxu9gvj\\pandas_69f0b32bc18b4ef0bc0e8b493c55bde0\\setup.py'"'"'; __file__='"'"'C:\\Users\\RVHS-D&T\\AppData\\Local\\Temp\\pip-install-rpxu9gvj\\pandas_69f0b32bc18b4ef0bc0e8b493c55bde0\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\RVHS-D&T\AppData\Local\Temp\pip-wheel-axk8bcxo'
       cwd: C:\Users\RVHS-D&T\AppData\Local\Temp\pip-install-rpxu9gvj\pandas_69f0b32bc18b4ef0bc0e8b493c55bde0\
  Complete output (841 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-3.8\pandas
  copying pa

In [ ]:
!pip install matplotlib

In [1]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os
import string

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cur_dir = os.path.abspath(os.path.dirname('__file__'))
cur_dir = os.path.join(cur_dir, 'drive')
cur_dir = os.path.join(cur_dir, 'MyDrive')
cur_dir = os.path.join(cur_dir,  '2021SMP')
cur_dir = os.path.join(cur_dir, 'SMP 21RV03')
cur_dir = os.path.join(cur_dir, 'Server Model')
cur_dir = os.path.join(cur_dir,  'Data')
filename = os.path.join(cur_dir, 'new_data.txt')
file = open(filename, "r")

In [4]:
length = 91165
counter = 0
idx = 0
texts = []
new_texts = []
for row in file:
    texts.append(row)
#print(texts[:100])

for text in texts:
    text = text.replace("\n", "").replace("\r", "").replace('\ufeff', '')
    punctuation_to_space = str.maketrans(string.punctuation, " "*len(string.punctuation))
    new_text = text.translate(punctuation_to_space)
    new_texts.append(new_text)
    counter += 1
    print("{}%".format(counter/length * 100), end="\r")

data = ""
counter = 0
for new in new_texts:
    try:
      if new[-1] == " ":
        data += new
      else:
        data += new + " "
    except:
      data += new + " "
    counter += 1
    print("{}%".format(counter/length * 100), end="\r")
print(data[:1000])

‎image omitte ‎this message was deleted ‎you deleted this message corpus preprocessing ipynb ‎document omitte keyboard software ipynb ‎document omitte and this is where the model can be teste yea the merged model would have the same layer parameter is it better if we include the ending punctuations  such as full stop  so that the model can detect if it is the end of the sentence the school may require your parents to be called on short notice and we need all phone numbers to be updated  both yours and your parents to all cce reps make sure you have collected the straits times special edition for cce lesson at 1100  collect before that good morning 3is chong su ying back in s good afternoon  any of you are given shn from ica  please reply to me personally with the below details any of you have parents or people in your household who travelled recently ‎image omitte  schedule for today is caiyi in class today for household members issue shn  student can come to school  but observe robust

In [ ]:
# tokenize data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# convert to tf-idf vector form
pickle.dump(tokenizer, open(os.path.join(cur_dir, 'tokenizer.pkl'), 'wb'))
sequence_data = tokenizer.texts_to_sequences([data])[0]
#
data = None
vocab_size = len(tokenizer.word_index)+1
sequences = []
for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)

sequences = np.array(sequences)
x = []
y = []
for i in sequences:
    x.append(i[0])
    y.append(i[1])
print("Done")
#
sequences = None
x = np.array(x)
y = np.array(y)
print("Done")
y = to_categorical(y, num_classes=vocab_size)
print("Done")
print(x.shape, y.shape)

Done
Done


In [ ]:
# Sequential Model (DO NOT RUN ANYMORE)
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1)) # sequential layer
model.add(LSTM(1000, return_sequences=True)) # first LSTM layer
model.add(LSTM(1000)) # second LSTM layer
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax")) # output layer + calculate probabilities

model.summary()

! Change RunTime to GPU


In [ ]:
### Change RunTime to GPU ###

# Callbacks
checkpoint = ModelCheckpoint(os.path.join(cur_dir, "server_model.h5"), monitor="loss", verbose=1,\
                            save_best_only=True, mode="auto")
# stores weight of model after training
# best model would be saved in "nextword1.h5"

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.00001, verbose = 1)
# reduce learning rate after a specified number of epochs
# in this case, if accuracy does not increase after 3 epochs, learning rate will decrease by 0.2

logdir='logsnextword'
tensorboard_Visualization = TensorBoard(log_dir=logdir)
# plot graph for visualization

# Compile Model
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.01), metrics=["accuracy"])
history = model.fit(x, y, epochs=40, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])
# epoch=150

plt.plot(history.history['loss'])
plt.show()